In [1]:
import requests
import re
import json

from sqlalchemy import create_engine, text as txt
from sqlalchemy.engine import URL
from sqlalchemy.orm import sessionmaker
import pymysql as mysql

host = "localhost"; db_user = "root"; psd = "Walter@254"; port = 3306; dbName = "pims"; #Replace these with the actual database credentials.
#host = "10.50.80.61"; db_user = "wadhawo"; psd = "Walter123"; port = 3306; dbName = "pims";

#Using PyMySQL
con=mysql.connect(host=host, user=db_user, password=psd, port=port, db=dbName)
cursor=con.cursor(); cursor.close(); con.close(); #Close PyMyQL since we'll use MySQLAlchemy engine
url_object = URL.create("mysql+pymysql", username=db_user, password=psd, host=host, database=dbName,)
engine = create_engine(url_object, pool_pre_ping=True, pool_recycle=3600)

# Define the URL and payload
url = "http://localhost/api/cpims/index.php"
#url = "http://10.50.80.61/api/cpims/index.php"
#url = 'http://localhost/database/server.php'
#url = 'http://10.50.80.61/database/server.php'

In [2]:
def send_post_request(url, payload):
    # Define headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    try:
        # Send POST request
        response = requests.post(url, data=payload, headers=headers)
        
        # Print response content
        print(response.text)
        
        # Check if the request was successful
        if response.status_code == 200:
            print("Request was successful.")
            # Perform further actions if needed
            # ...
        else:
            print(f"Request was not successful. Status code: {response.status_code} - {response.reason}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


In [3]:
import pandas as pd
import easygui

form1a = pd.read_excel(easygui.fileopenbox("Open XLSX file with OVC services"), sheet_name="Sheet6")
#form1a=pd.read_sql_query(sql=txt("SELECT * FROM form1a WHERE deleted IS NULL AND service LIKE 'CP %' AND (dos > DATE('2023-03-31') AND dos < DATE('2023-10-01'))"), con=engine.connect())
print("Got {} Form1A records.".format(form1a.shape[0]))


Got 238 Form1A records.


In [4]:
#Get RegList IDs
#reglist_ids=pd.read_sql_query(sql=txt("SELECT cpims_data.caregiver_id, cpims_ovc_id FROM cpims_data WHERE exit_status = 'ACTIVE'"), con=engine.connect())
#reglist_ids.reset_index(drop=True, inplace=True)
#print("Got {} Active OVC CPIMS IDs.".format(reglist_ids.shape[0]))

#MIS-CPIMS Mappings
domain_id = {'Schooled': 'DEDU', 'Healthy': 'DHNU', 'Stable': 'DHES', 'Safe': 'DPRO'}
cpims_response_mapping = {'Yes': 'AYES', 'No': 'ANNO', 'N/A': 'ANNA'}
f1a_service_id = { 'CP 1.2 HEs': 'CP12HEs', 'CP 1.3 HEs': 'CP13HEs', 'CP 2.1 HEs': 'CP21HEs', 'CP 2.11b HEs': 'CP211bHEs', 'CP 2.12 HEs': 'CP212HEs', 'CP 2.13b HEs': 'CP213bHEs', 'CP 2.2b HEs': 'CP22bHEs', 'CP 2.5 HEs': 'CP25HEs', 'CP 2.6 HEs': 'CP26HEs', 'CP 2.7 HEs': 'CP27HEs', 'CP 2.8 HEs': 'CP28HEs', 'CP 2.9 HEs': 'CP29HEs', 'CP 3.1 HEs': 'CP31HEs', 'CP 3.3 HEs': 'CP33HEs', 'CP 4.1a HEs': 'CP41aHEs', 'CP 4.1b HEs': 'CP41bHEs', 'CP 4.2 HEs': 'CP42HEs', 'CP 4.3 HEs': 'CP43HEs', 'CP 4.4b HEs': 'CP44bHEs', 'CP 4.5b HEs': 'CP45bHEs', 'CP 4.6b HEs': 'CP46bHEs', 'CP 5.10b STs': 'CP510bSTs', 'CP 5.3b STs': 'CP53bSTs', 'CP 5.5 STs': 'CP55STs', 'CP 5.7b STs': 'CP57bSTs', 'CP 5.8b STs': 'CP58bSTs', 'CP 5.9b STs': 'CP59bSTs', 'CP 6.1 SAs': 'CP61SAs', 'CP 6.3b SAs': 'CP63bSAs', 'CP 6.4 SAs': 'CP64SAs', 'CP 6.6 SAs': 'CP66SAs', 'CP 7.1b SAs': 'CP71bSAs', 'CP 8.1b SAs': 'CP81aSAs', 'CP 9.1 SCs': 'CP91SCs', 'CP 9.10 SCs': 'CP99SCs', 'CP 9.11 SCs': 'CP910SCs', 'CP 9.2 SCs': 'CP92SCs', 'CP 9.3 SCs': 'CP93SCs', 'CP 9.4a SCs': 'CP94aSCs', 'CP 9.8b SCs': 'CP911bSCs', 'CP 9.9b SCs': 'CP98bSCs' }
f1b_service_id = { 'CP 1.2 HEGs': 'CP12HEGs', 'CP 2.1 HEGs': 'CP21HEGs', 'CP 2.1 HEs': 'CP21HEGs', 'CP 2.10 HEGs': 'CP210HEGs', 'CP 2.11b HEGs': 'CP211bHEGs', 'CP 2.13b HEGs': 'CP213bHEGs', 'CP 2.2b HEGs': 'CP22bHEGs', 'CP 2.5 HEGs': 'CP25HEGs', 'CP 2.6 HEGs': 'CP26HEGs', 'CP 2.7 HEGs': 'CP27HEGs', 'CP 2.9 HEGs': 'CP29HEGs', 'CP 4.5b HEGs': 'CP45bHEGs', 'CP 4.6b HEGs': 'CP46bHEGs', 'CP 4.7 HEGs': 'CP47HEGs', 'CP 5.1 STGs': 'CP51STGs', 'CP 5.12b STGs': 'CP512bSTGs', 'CP 5.2b STGs': 'CP52bSTGs', 'CP 5.3b STGs': 'CP53bSTGs', 'CP 5.3d STGs': 'CP53dSTGs', 'CP 5.4b STGs': 'CP54bSTGs', 'CP 5.5 STGs': 'CP55STGs', 'CP 5.6 STGs': 'CP56STGs', 'CP 5.7b STGs': 'CP57bSTGs', 'CP 5.8b STGs': 'CP58bSTGs', 'CP 5.9b STGs': 'CP59bSTGs', 'CP 6.1 SAGs': 'CP61SAGs', 'CP 6.10b SAGs': 'CP610bSAGs', 'CP 6.11b SAGs': 'CP611bSAGs', 'CP 6.2 SAGs': 'CP62SAGs', 'CP 6.5 SAGs': 'CP6.5SAGs', 'CP 6.8 SAGs': 'CP68SAGs', 'CP 6.9b SAGs': 'CP69bSAGs' }
cp_goals_id = { 'GOAL 1': 'GH1HE', 'GOAL 2': 'GH2HE', 'GOAL 3': 'GH3HE', 'GOAL 4': 'GH4HE', 'GOAL 5': 'GH5ST', 'GOAL 6': 'GH6SA', 'GOAL 7': 'GH7SA', 'GOAL 8': 'GH8SA', 'GOAL 9': 'GH9SC' }
cp_gap_id = { 'GN 1.1 HE': 'GN11HE', 'GN 1.2 HE': 'GN12HE', 'GN 1.3 HE': 'GN13HE', 'GN 2.1 HE': 'GN21HE', 'GN 2.10 HE': 'GN210H', 'GN 2.2 HE': 'GN22HE', 'GN 2.3 HE': 'GN23HE', 'GN 2.4 HE': 'GN24HE', 'GN 2.5 HE': 'GN25HE', 'GN 2.6 HE': 'GN26HE', 'GN 2.7 HE': 'GN27HE', 'GN 2.8 HE': 'GN28HE', 'GN 2.9 HE': 'GN29HE', 'GN 3.1 HE': 'GN31HE', 'GN 3.2 HE': 'GN32HE', 'GN 3.3 HE': 'GN33HE', 'GN 3.4 HE': 'GN34HE', 'GN 4.1 HE': 'GN41HE', 'GN 4.2 HE': 'GN42HE', 'GN 4.3 HE': 'GN43HE', 'GN 4.4 HE': 'GN44HE', 'GN 4.5 HE': 'GN45HE', 'GN 5.1 ST': 'GN51ST', 'GN 5.2 ST': 'GN52ST', 'GN 5.3 ST': 'GN53ST', 'GN 6.1 SA': 'GN61SA', 'GN 6.2 SA': 'GN62SA', 'GN 6.3 SA': 'GN63SA', 'GN 7.1 SA': 'GN71SA', 'GN 8.1 SA': 'GN81SA', 'GN 9.1 SC': 'GN91SC', 'GN 9.2 SC': 'GN92SC', 'GN 9.3 SC': 'GN93SC', 'GN 9.4 SC': 'GN94SC', 'GN 9.5 SC': 'GN95SC' }
cp_priority_id = { 'P 1.1 HE': 'P11HE', 'P 1.2 HE': 'P12HE', 'P 1.3 HE': 'P13HE', 'P 1.6 ST': 'P16ST', 'P 1.7 ST': 'P17ST', 'P 2.1 HE': 'P21HE', 'P 2.2 HE': 'P22HE', 'P 2.3 HE': 'P23HE', 'P 2.4 HE': 'P24HE', 'P 2.5 HE': 'P25HE', 'P 2.6 HE': 'P26HE', 'P 3.1 HE': 'P31HE', 'P 3.2 HE': 'P32HE', 'P 3.3 HE': 'P33HE', 'P 4.1 HE': 'P41HE', 'P 4.2 HE': 'P42HE', 'P 4.3 HE': 'P43HE', 'P 4.4 HE': 'P44HE', 'P 4.5 HE': 'P45HE', 'P 5.1 ST': 'P51ST', 'P 5.2 ST': 'P52ST', 'P 5.3 ST': 'P53ST', 'P 5.4 ST': 'P54ST', 'P 5.5 ST': 'P55ST', 'P 5.6 ST': 'P16ST', 'P 5.7 ST': 'P17ST', 'P 6.1 SA': 'P61SA', 'P 6.2 SA': 'P62SA', 'P 6.3 SA': 'P63SA', 'P 7.1 SA': 'P71SA', 'P 8.1 SA': 'P81SA', 'P 9.1 SC': 'P91SC', 'P 9.2 SC': 'P92SC', 'P 9.3 SC': 'P93SC', 'P 9.4 SC': 'P94SC', 'P 9.5 SC': 'P95SC', 'P 9.6 SC': 'P96SC', 'P 9.7 SC': 'P97SC'}
cp_service_id = { 'CP 1.1 HE': 'CP11HE', 'CP 1.2 HE': 'CP12 HE', 'CP 1.3 HE': 'CP13HE', 'CP 1.4 HE': 'CP14HE', 'CP 2.1 HE': 'CP21HE', 'CP 2.10 HE': 'CP210HE', 'CP 2.11 HE': 'CP211HE', 'CP 2.12 HE': 'CP212HE', 'CP 2.13 HE': 'CP213HE', 'CP 2.2 HE': 'CP22HE', 'CP 2.3 HE': 'CP23HE', 'CP 2.4 HE': 'CP24HE', 'CP 2.5 HE': 'CP25HE', 'CP 2.6 HE': 'CP26HE', 'CP 2.7 HE': 'CP27HE', 'CP 2.8 HE': 'CP28HE', 'CP 2.9 HE': 'CP29HE', 'CP 3.1 HE': 'CP31HE', 'CP 3.2 HE': 'CP32HE', 'CP 4.1 HE': 'CP41HE', 'CP 4.2 HE': 'CP42HE', 'CP 4.3 HE': 'CP43HE', 'CP 4.4 HE': 'CP44HE', 'CP 4.5 HE': 'CP45HE', 'CP 4.6 HE': 'CP46HE', 'CP 4.7 HE': 'CP47HE', 'CP 5.1 ST': 'CP51ST', 'CP 5.10 ST': 'CP510S', 'CP 5.11 ST': 'CP511S', 'CP 5.12 ST': 'CP512S', 'CP 5.13 ST': 'CP513S', 'CP 5.2 ST': 'CP52ST', 'CP 5.3 ST': 'CP53ST', 'CP 5.4 ST': 'CP54ST', 'CP 5.5 ST': 'CP55ST', 'CP 5.6 ST': 'CP56ST', 'CP 5.7 ST': 'CP57ST', 'CP 5.8 ST': 'CP58ST', 'CP 5.9 ST': 'CP59ST', 'CP 6.1 SA': 'CP61SA', 'CP 6.10 SA': 'CP610S', 'CP 6.11 SA': 'CP611S', 'CP 6.2 SA': 'CP62SA', 'CP 6.3 SA': 'CP63SA', 'CP 6.4 SA': 'CP64SA', 'CP 6.5 SA': 'CP65SA', 'CP 6.6 SA': 'CP66SA', 'CP 6.7 SA': 'CP67SA', 'CP 6.8 SA': 'CP68SA', 'CP 6.9 SA': 'CP69SA', 'CP 7.1 SA': 'CP71SA', 'CP 8.1 SA': 'CP81SA', 'CP 9.1 SC': 'CP91SC', 'CP 9.10 SC': 'CP99SC', 'CP 9.11 SC': 'CP910S', 'CP 9.2 SC': 'CP92SC', 'CP 9.3 SC': 'CP93SC', 'CP 9.4 SC': 'CP94SC', 'CP 9.5 SC': 'CP95SC', 'CP 9.6 SC': 'CP96SC', 'CP 9.7 SC': 'CP97SC', 'CP 9.9 SC': 'CP98SC' }
cpara_questions_mapping = { "child_headed": "CP3d", "has_hei": "CP4d", "has_pbf": "CP5d", "q1_1": "CP1q", "q1_2": "CP2q", "q1_3": "CP3q", "q1_4": "CP4q", "q2_1": "CP6q", "q2_2": "CP7q", "q2_3": "CP8q", "q2_4": "CP9q", "q2_5": "CP10q", "q2_6": "CP11q", "q2_8": "CP12q", "q2_7": "CP13q", "q2_9": "CP14q", "q3_1": "CP15q", "q3_2": "CP16q", "q3_3": "CP17q", "q4_1": "CP18q", "q4_2": "CP19q", "q4_3": "CP20q", "q4_4": "CP21q", "q5_1": "CP22q", "q5_2": "CP23q", "q5_3": "CP24q", "q6_1": "CP25q", "q6_2": "CP26q", "q6_3": "CP27q", "q6_4": "CP28q", "q6_5": "CP29q", "q7_1": "CP30q", "q7_2": "CP31q", "q8_1": "CP32q", "q9_1": "CP33q", "q9_2": "CP34q", "q9_3": "CP35q", "q9_4": "CP36q", "benchmark1": "CP1b", "benchmark2": "CP2b", "benchmark3": "CP3b", "benchmark4": "CP4b", "benchmark5": "CP5b", "benchmark6": "CP6b", "benchmark7": "CP7b", "benchmark8": "CP8b", "benchmark9": "CP9b" }
cpims_response_mapping = {'Yes':'AYES', 'No':'ANNO', 'N/A':'ANNA'}

form1a[["ovc_id", "service", "dos", "domain"]]

,ovc_id,service,dos,domain
0,1712259,CP23HEs,2023-12-20,Healthy
1,3496748,CP23HEs,2023-12-20,Healthy
2,1933076,CP23HEs,2023-12-20,Healthy
3,4338845,CP23HEs,2023-12-20,Healthy
4,4045577,CP23HEs,2023-12-20,Healthy
...,...,...,...,...
233,3694849,CP81aSAs,2023-12-20,Safe
234,3695206,CP81aSAs,2023-12-21,Safe
235,3697564,CP81aSAs,2023-12-22,Safe
236,4044347,CP81aSAs,2023-12-23,Safe


In [5]:
#Submit form1A
def pushForm1A():
    df = form1a
    counter = 0
    
    for row in df.itertuples(index=False):
        if(len(row.service)>0):
            # Define headers
            headers = {
                'Content-Type': 'application/x-www-form-urlencoded'
            }

            payload = {
                "formType": "form1a",
                "ovc_id": row.ovc_id,
                "dos": row.dos, #.strftime('%Y-%m-%d'),
                "domain_id": domain_id.get(row.domain, ''),
                "service_id": row.service
            }
            
            try:
                # Send POST request
                response = requests.post(url, data=payload, headers=headers)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check if the response is in JSON format
                    if 'application/json' in response.headers.get('content-type', '').lower():
                        # Parse and print JSON response
                        response_json = response.json()
                        print("JSON Response:", response_json)
                    else:
                        counter += 1
                        response_text = response.text.replace('\\', '').replace('{', '').replace('}','').replace('"','')
                        script_output = {
                            "mis_response" : "Form1A CPIMS Sync",
                            "cpims_response" : {response_text.split(":")[0] : response_text.split(":")[1]},
                            "payload" : {"cpims_id": row.ovc_id, "date_of_service": row.dos, "service": row.service}, #payload,
                            "record_id": row.record_id,
                            "record_count" : str(counter)+"/"+str(form1a.shape[0])
                        }
                        # If not JSON, print the response as text
                        print("Script Response:", script_output)
                else:
                    print(f"Request was not successful. Status code: {response.status_code} - {response.reason}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print("This Form1A service is empty - record_id={}, service={}".format(row.record_id, row.service))
            
#pushForm1A()

In [6]:
import traceback
def main():
    try:
        pushForm1A()
    except Exception as e:
        # Print the error, but continue running
        print(f"An error occurred: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

Script Response: {'mis_response': 'Form1A CPIMS Sync', 'cpims_response': {'details': 'Form F1A details saved successfully'}, 'payload': {'cpims_id': 1712259, 'date_of_service': '2023-12-20', 'service': 'CP23HEs'}, 'record_id': 1, 'record_count': '1/238'}
Script Response: {'mis_response': 'Form1A CPIMS Sync', 'cpims_response': {'details': 'Form F1A details saved successfully'}, 'payload': {'cpims_id': 3496748, 'date_of_service': '2023-12-20', 'service': 'CP23HEs'}, 'record_id': 2, 'record_count': '2/238'}
Script Response: {'mis_response': 'Form1A CPIMS Sync', 'cpims_response': {'details': 'Form F1A details saved successfully'}, 'payload': {'cpims_id': 1933076, 'date_of_service': '2023-12-20', 'service': 'CP23HEs'}, 'record_id': 3, 'record_count': '3/238'}
Script Response: {'mis_response': 'Form1A CPIMS Sync', 'cpims_response': {'details': 'Form F1A details saved successfully'}, 'payload': {'cpims_id': 4338845, 'date_of_service': '2023-12-20', 'service': 'CP23HEs'}, 'record_id': 4, 'reco